In [2]:
# Load the libraries
import numpy as np 
import pandas as pd 
import seaborn as sns
import re
from matplotlib import pyplot as plt
import matplotlib.pyplot as plt
from warnings import filterwarnings
filterwarnings('ignore')

from scipy.stats.mstats import winsorize
import scipy.stats as stats
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error,explained_variance_score
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import (GridSearchCV, cross_val_score, cross_val_predict, StratifiedKFold, learning_curve)
from sklearn.metrics import confusion_matrix, classification_report

from statsmodels.tools.eval_measures import mse, rmse
from sklearn import preprocessing
## Display all the columns of the dataframe
pd.pandas.set_option('display.max_columns', None)

In [8]:
df=pd.read_csv("PRT_Quarterly_Employment_By_Occupation.csv")
df.head(2)

,ref_area.label,indicator.label,source.label,sex.label,classif1.label,classif2.label,time,obs_value,obs_status.label,note_classif.label,note_indicator.label,note_source.label
0,Portugal,Employees by sex and economic activity (thousa...,LFS - EU Labour Force Survey,Sex: Male,Economic activity (Aggregate): Construction,NaN,2020Q4,185.8,NaN,NaN,Frequency: Quarterly,NaN
1,Portugal,Employees by sex and economic activity (thousa...,LFS - EU Labour Force Survey,Sex: Female,Economic activity (Aggregate): Construction,NaN,2020Q4,23.4,NaN,NaN,Frequency: Quarterly,NaN


In [9]:
df.tail(2)

,ref_area.label,indicator.label,source.label,sex.label,classif1.label,classif2.label,time,obs_value,obs_status.label,note_classif.label,note_indicator.label,note_source.label
486,Portugal,Employees by sex and occupation (thousands),LFS - EU Labour Force Survey,Sex: Male,Occupation (ISCO-08): 4. Clerical support workers,NaN,2011Q1,145.3,Break in series,NaN,Frequency: Quarterly | Break in series: Method...,NaN
487,Portugal,Employees by sex and occupation (thousands),LFS - EU Labour Force Survey,Sex: Female,Occupation (ISCO-08): 4. Clerical support workers,NaN,2011Q1,259.9,Break in series,NaN,Frequency: Quarterly | Break in series: Method...,NaN


In [10]:
df.shape

(488, 12)

In [11]:
df.isnull().sum()

ref_area.label            0
indicator.label           0
source.label              0
sex.label                 0
classif1.label            0
classif2.label          488
time                      0
obs_value                 0
obs_status.label        474
note_classif.label      488
note_indicator.label      0
note_source.label       488
dtype: int64

In [12]:
df=df[["sex.label", "classif1.label", "time", "obs_value"]]

In [13]:
df.head()

,sex.label,classif1.label,time,obs_value
0,Sex: Male,Economic activity (Aggregate): Construction,2020Q4,185.8
1,Sex: Female,Economic activity (Aggregate): Construction,2020Q4,23.4
2,Sex: Male,Economic activity (Aggregate): Construction,2020Q3,203.3
3,Sex: Female,Economic activity (Aggregate): Construction,2020Q3,23.1
4,Sex: Male,Economic activity (Aggregate): Construction,2020Q2,206.0


In [14]:
df["sex.label"].unique()

array(['Sex: Male', 'Sex: Female'], dtype=object)

In [15]:
df["classif1.label"].unique()

array(['Economic activity (Aggregate): Construction',
       'Occupation (Skill level): Skill levels 3 and 4 ~ high',
       'Occupation (Skill level): Skill level 2 ~ medium',
       'Occupation (Skill level): Skill level 1 ~ low',
       'Occupation (ISCO-08): 4. Clerical support workers'], dtype=object)

In [16]:
df.tail()

,sex.label,classif1.label,time,obs_value
483,Sex: Female,Occupation (Skill level): Skill levels 3 and 4...,2011Q1,545.1
484,Sex: Female,Occupation (Skill level): Skill level 2 ~ medium,2011Q1,918.8
485,Sex: Female,Occupation (Skill level): Skill level 1 ~ low,2011Q1,395.0
486,Sex: Male,Occupation (ISCO-08): 4. Clerical support workers,2011Q1,145.3
487,Sex: Female,Occupation (ISCO-08): 4. Clerical support workers,2011Q1,259.9


In [17]:
df['gender'] = df['sex.label'].str.extract('(Male|Female)')

In [18]:
df['male'] = df['gender'].apply(lambda x: 1 if x == 'Male' else 0)
df['female'] = df['gender'].apply(lambda x: 1 if x == 'Female' else 0)

In [19]:
df.drop(columns=['sex.label'], inplace=True)

In [20]:
df.head()

,classif1.label,time,obs_value,gender,male,female
0,Economic activity (Aggregate): Construction,2020Q4,185.8,Male,1,0
1,Economic activity (Aggregate): Construction,2020Q4,23.4,Female,0,1
2,Economic activity (Aggregate): Construction,2020Q3,203.3,Male,1,0
3,Economic activity (Aggregate): Construction,2020Q3,23.1,Female,0,1
4,Economic activity (Aggregate): Construction,2020Q2,206.0,Male,1,0


In [21]:
df["classif1.label"].unique()

array(['Economic activity (Aggregate): Construction',
       'Occupation (Skill level): Skill levels 3 and 4 ~ high',
       'Occupation (Skill level): Skill level 2 ~ medium',
       'Occupation (Skill level): Skill level 1 ~ low',
       'Occupation (ISCO-08): 4. Clerical support workers'], dtype=object)

In [22]:
new_df = pd.DataFrame()

# create a column for each label, filled with zeros
new_df['Construction'] = 0
new_df['Clerical support workers'] = 0
new_df['Skill levels 3 and 4 ~ high'] = 0
new_df['Skill level 2 ~ medium'] = 0
new_df['Skill level 1 ~ low'] = 0


# loop through each row of the original dataframe
for index, row in df.iterrows():

    # get the label from the current row
    label = row['classif1.label']

    # set the value in the appropriate new column to 1
    if label == 'Occupation (ISCO-08): 4. Clerical support workers':
        new_df.at[index, 'Clerical support workers'] = 1
    elif label == 'Economic activity (Aggregate): Construction':
        new_df.at[index, 'Construction'] = 1
    elif label == 'Occupation (Skill level): Skill levels 3 and 4 ~ high':
        new_df.at[index, 'Skill levels 3 and 4 ~ high'] = 1
    elif label == 'Occupation (Skill level): Skill level 2 ~ medium':
        new_df.at[index, 'Skill level 2 ~ medium'] = 1
    elif label == 'Occupation (Skill level): Skill level 1 ~ low':
        new_df.at[index, 'Skill level 1 ~ low'] = 1

# concatenate the original dataframe and the new dataframe
result_df = pd.concat([df, new_df], axis=1)

# print the resulting dataframe
result_df.head()


,classif1.label,time,obs_value,gender,male,female,Construction,Clerical support workers,Skill levels 3 and 4 ~ high,Skill level 2 ~ medium,Skill level 1 ~ low
0,Economic activity (Aggregate): Construction,2020Q4,185.8,Male,1,0,1.0,NaN,NaN,NaN,NaN
1,Economic activity (Aggregate): Construction,2020Q4,23.4,Female,0,1,1.0,NaN,NaN,NaN,NaN
2,Economic activity (Aggregate): Construction,2020Q3,203.3,Male,1,0,1.0,NaN,NaN,NaN,NaN
3,Economic activity (Aggregate): Construction,2020Q3,23.1,Female,0,1,1.0,NaN,NaN,NaN,NaN
4,Economic activity (Aggregate): Construction,2020Q2,206.0,Male,1,0,1.0,NaN,NaN,NaN,NaN


In [23]:
# drop the original 'classif1.label' column
result_df = result_df.drop(['classif1.label'], axis=1)

In [24]:
result_df.isnull().sum()

time                             0
obs_value                        0
gender                           0
male                             0
female                           0
Construction                   320
Clerical support workers       408
Skill levels 3 and 4 ~ high    408
Skill level 2 ~ medium         408
Skill level 1 ~ low            408
dtype: int64

In [25]:
result_df.shape

(488, 10)

In [26]:
result_df.fillna(0, inplace=True)

In [29]:
#result_df=result_df.drop(["Construction","gender"],axis=1)

In [30]:
result_df.tail()

,time,obs_value,gender,male,female,Construction,Clerical support workers,Skill levels 3 and 4 ~ high,Skill level 2 ~ medium,Skill level 1 ~ low
483,2011Q1,545.1,Female,0,1,0.0,0.0,1.0,0.0,0.0
484,2011Q1,918.8,Female,0,1,0.0,0.0,0.0,1.0,0.0
485,2011Q1,395.0,Female,0,1,0.0,0.0,0.0,0.0,1.0
486,2011Q1,145.3,Male,1,0,0.0,1.0,0.0,0.0,0.0
487,2011Q1,259.9,Female,0,1,0.0,1.0,0.0,0.0,0.0


In [31]:
result_df.head()

,time,obs_value,gender,male,female,Construction,Clerical support workers,Skill levels 3 and 4 ~ high,Skill level 2 ~ medium,Skill level 1 ~ low
0,2020Q4,185.8,Male,1,0,1.0,0.0,0.0,0.0,0.0
1,2020Q4,23.4,Female,0,1,1.0,0.0,0.0,0.0,0.0
2,2020Q3,203.3,Male,1,0,1.0,0.0,0.0,0.0,0.0
3,2020Q3,23.1,Female,0,1,1.0,0.0,0.0,0.0,0.0
4,2020Q2,206.0,Male,1,0,1.0,0.0,0.0,0.0,0.0


In [32]:
df=result_df.rename(columns={'time':'Year_Quarter', 'obs_value':'Number_of_employees', 'Clerical support workers':'Clerical_support_workers',
                      'Skill levels 3 and 4 ~ high':'Skill_levels_high', 'Skill level 2 ~ medium':'Skill_level_medium',
                      'Skill level 1 ~ low':'Skill_level_low'})

In [33]:
df.tail()

,Year_Quarter,Number_of_employees,gender,male,female,Construction,Clerical_support_workers,Skill_levels_high,Skill_level_medium,Skill_level_low
483,2011Q1,545.1,Female,0,1,0.0,0.0,1.0,0.0,0.0
484,2011Q1,918.8,Female,0,1,0.0,0.0,0.0,1.0,0.0
485,2011Q1,395.0,Female,0,1,0.0,0.0,0.0,0.0,1.0
486,2011Q1,145.3,Male,1,0,0.0,1.0,0.0,0.0,0.0
487,2011Q1,259.9,Female,0,1,0.0,1.0,0.0,0.0,0.0


In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 488 entries, 0 to 487
Data columns (total 10 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Year_Quarter              488 non-null    object 
 1   Number_of_employees       488 non-null    float64
 2   gender                    488 non-null    object 
 3   male                      488 non-null    int64  
 4   female                    488 non-null    int64  
 5   Construction              488 non-null    float64
 6   Clerical_support_workers  488 non-null    float64
 7   Skill_levels_high         488 non-null    float64
 8   Skill_level_medium        488 non-null    float64
 9   Skill_level_low           488 non-null    float64
dtypes: float64(6), int64(2), object(2)
memory usage: 38.2+ KB


In [40]:
df["Number_of_employees"] = df["Number_of_employees"].astype('int')
df["Construction"] = df["Construction"].astype('int')
df["Clerical_support_workers"] = df["Clerical_support_workers"].astype('int')
df["Skill_levels_high"] = df["Skill_levels_high"].astype('int')
df["Skill_level_medium"] = df["Skill_level_medium"].astype('int')
df["Skill_level_low"] = df["Skill_level_low"].astype('int')

In [41]:
df.head()

,Year_Quarter,Number_of_employees,gender,male,female,Construction,Clerical_support_workers,Skill_levels_high,Skill_level_medium,Skill_level_low
0,2020Q4,185,Male,1,0,1,0,0,0,0
1,2020Q4,23,Female,0,1,1,0,0,0,0
2,2020Q3,203,Male,1,0,1,0,0,0,0
3,2020Q3,23,Female,0,1,1,0,0,0,0
4,2020Q2,206,Male,1,0,1,0,0,0,0


In [103]:
portugal_ml = df.to_csv('portugal_ml.csv', index=False)
#df.to_csv("Portugal_ml", index=False)

## Statistics for Data Analytics Tasks

## 1. T-Test One Population - Number_of_employees in Construction industry in Portugal.

In [42]:
df["Number_of_employees"].mean()

394.8032786885246

In [43]:
X= df["Number_of_employees"];X

0      185
1       23
2      203
3       23
4      206
      ... 
483    545
484    918
485    395
486    145
487    259
Name: Number_of_employees, Length: 488, dtype: int32

In [44]:
#H0 : u = 395
#H1 : u =! 395
stats.ttest_1samp(X,395)

Ttest_1sampResult(statistic=-0.012853466621920826, pvalue=0.9897499635653716)

In [45]:
##  statistic=0.016522005465077535 is the t value = 0.01

In [46]:
##  if p-value < 0.05  we reject Ho
## if p-value > 0.05 we accept tht Ho 
## the p- vlaue is 0.986 is greater than 0.05 so we accept Ho

In [47]:
## We accept H0 ---> so there is enough evidence to say that the average Number_of_employees 395,Quarterly.

## 2. T-Test - two populations Qarterly Number of employees in Q1 and Q4 

In [48]:
df1=df[['Year_Quarter','Number_of_employees']]

In [49]:
df1.tail()

,Year_Quarter,Number_of_employees
483,2011Q1,545
484,2011Q1,918
485,2011Q1,395
486,2011Q1,145
487,2011Q1,259


In [50]:
#We group the data according to the categories requested.
Q1 = df1.query('Year_Quarter == "2020Q1"')['Number_of_employees']
Q4 = df1.query('Year_Quarter == "2020Q4"')['Number_of_employees']

In [51]:
#We perform the test assuming as H0 that mu1 = mu2 (Q1 Number_of_employees = Q4 Number_of_employees)
test = stats.ttest_ind(Q1, Q4, equal_var=True)
display(test)

Ttest_indResult(statistic=0.015542210227977334, pvalue=0.9877705870949717)

In [52]:
## pvalue=0.98 >alpha(0.05) means we accept the H0, there is no significant difference in Q1 Number_of_employees and Q4 Number_of_employees.

In [53]:
#We group the data according to the categories requested.
Q1 = df1.query('Year_Quarter == "2011Q1"')['Number_of_employees']
Q4 = df1.query('Year_Quarter == "2011Q4"')['Number_of_employees']

In [54]:
test = stats.ttest_ind(Q1, Q4, equal_var=True)
display(test)

Ttest_indResult(statistic=0.09300855183217407, pvalue=0.9269241748459716)

In [55]:
 ## p-value 0.92 is > alpha 0.05 so we accept the H0,

## 3. Confidence Intervel 

In [56]:
import scipy.stats as st

In [57]:
df.head()

,Year_Quarter,Number_of_employees,gender,male,female,Construction,Clerical_support_workers,Skill_levels_high,Skill_level_medium,Skill_level_low
0,2020Q4,185,Male,1,0,1,0,0,0,0
1,2020Q4,23,Female,0,1,1,0,0,0,0
2,2020Q3,203,Male,1,0,1,0,0,0,0
3,2020Q3,23,Female,0,1,1,0,0,0,0
4,2020Q2,206,Male,1,0,1,0,0,0,0


In [58]:
df["Number_of_employees"].describe()

count     488.000000
mean      394.803279
std       338.096790
min        12.000000
25%       131.000000
50%       284.000000
75%       649.750000
max      1199.000000
Name: Number_of_employees, dtype: float64

In [59]:
#DEFINE THE VARIABLE WE FIND THE CONFIDENCE INTERVEL OF Number_of_employees.
X = df.iloc[:, 1:2].values
X

array([[ 185],
       [  23],
       [ 203],
       [  23],
       [ 206],
       [  14],
       [ 220],
       [  12],
       [ 208],
       [  15],
       [ 214],
       [  16],
       [ 213],
       [  18],
       [ 208],
       [  19],
       [ 207],
       [  19],
       [ 219],
       [  17],
       [ 223],
       [  19],
       [ 211],
       [  19],
       [ 215],
       [  19],
       [ 205],
       [  14],
       [ 211],
       [  17],
       [ 208],
       [  20],
       [ 209],
       [  18],
       [ 196],
       [  16],
       [ 199],
       [  17],
       [ 201],
       [  15],
       [ 200],
       [  17],
       [ 193],
       [  18],
       [ 196],
       [  12],
       [ 191],
       [  14],
       [ 195],
       [  15],
       [ 207],
       [  13],
       [ 192],
       [  13],
       [ 200],
       [  14],
       [ 206],
       [  15],
       [ 195],
       [  15],
       [ 207],
       [  14],
       [ 219],
       [  15],
       [ 216],
       [  15],
       [ 2

In [60]:
#CENTRAL TENDENCY FOR THE Number_of_employees
mean = X.mean()
print("The mean is ", mean)

The mean is  394.8032786885246


In [61]:
#create 90% confidence interval for population mean of Number_of_employees per Quater
st.t.interval(alpha=0.95, df=len(X)-1, loc=np.mean(X), scale=st.sem(X)) 

(array([364.73144594]), array([424.87511144]))

In [62]:
## The population average Number_of_employees is between 365 and 425.

## Chi Squared test

In [63]:
from numpy import sqrt, abs, round
import scipy.stats as stats
from scipy.stats import norm

In [64]:
df.head()

,Year_Quarter,Number_of_employees,gender,male,female,Construction,Clerical_support_workers,Skill_levels_high,Skill_level_medium,Skill_level_low
0,2020Q4,185,Male,1,0,1,0,0,0,0
1,2020Q4,23,Female,0,1,1,0,0,0,0
2,2020Q3,203,Male,1,0,1,0,0,0,0
3,2020Q3,23,Female,0,1,1,0,0,0,0
4,2020Q2,206,Male,1,0,1,0,0,0,0


In [65]:
df["female"].unique()

array([0, 1], dtype=int64)

In [66]:
df["Clerical_support_workers"].unique()

array([0, 1])

In [67]:
## Null Hypothesis: There is significant association between female and Clerical_support_workers features.

## Alternate Hypothesis: There is no significant association between female and Clerical_support_workers features.

In [68]:
dataset_table=pd.crosstab(df['female'],df['Clerical_support_workers'])
dataset_table

Clerical_support_workers,0,1
female,,
0,204,40
1,204,40


In [69]:
observed=dataset_table.values
val2=stats.chi2_contingency(dataset_table)
expected=val2[3]

In [70]:
alpha=0.5
ddof =1

In [71]:
from scipy.stats import chi2
chi_square=sum([(o-e)**2./e for o,e in zip(observed,expected)])
chi_square_statistic=chi_square[0]+chi_square[1]
p_value=1-chi2.cdf(x=chi_square_statistic,df=1)
     

In [72]:
print("chi-square statistic:-",chi_square_statistic)

print('Significance level: ',alpha)

print('Degree of Freedom: ',ddof)

print('p-value:',p_value)  

chi-square statistic:- 0.0
Significance level:  0.5
Degree of Freedom:  1
p-value: 1.0


In [73]:
if p_value<=alpha:
    print("Reject Null Hypothesis")
else:
    print("Accept Null Hypthesis")

Accept Null Hypthesis


In [74]:
## Since the p-value is greater than 0.05, we accept the null hypothesis. We conclude that There is significant association between female and Clerical_support_workers features.

## Non-Parametric Tests ## Need to split the notebook from here.

## 1. Sign Test

In [75]:
## Apply the sign test to compare the median Number_of_employees per Year_Quarter between the years 2011Q1 and 2020Q4.

In [76]:
## H0 : The median Number_of_employees per Year_Quarter in construction industry are the same for the years between 2011Q1 and 2020Q4.
## H1 : The median Number_of_employees per Year_Quarter in construction industry different for the years between 2011Q1 and 2020Q4.

In [77]:
df.head()

,Year_Quarter,Number_of_employees,gender,male,female,Construction,Clerical_support_workers,Skill_levels_high,Skill_level_medium,Skill_level_low
0,2020Q4,185,Male,1,0,1,0,0,0,0
1,2020Q4,23,Female,0,1,1,0,0,0,0
2,2020Q3,203,Male,1,0,1,0,0,0,0
3,2020Q3,23,Female,0,1,1,0,0,0,0
4,2020Q2,206,Male,1,0,1,0,0,0,0


In [78]:
# subset the data for years 2011Q1 and 2020Q4
df_subset = df[df["Year_Quarter"].isin(["2011Q1", "2020Q4"])]
df_subset

,Year_Quarter,Number_of_employees,gender,male,female,Construction,Clerical_support_workers,Skill_levels_high,Skill_level_medium,Skill_level_low
0,2020Q4,185,Male,1,0,1,0,0,0,0
1,2020Q4,23,Female,0,1,1,0,0,0,0
78,2011Q1,311,Male,1,0,1,0,0,0,0
79,2011Q1,22,Female,0,1,1,0,0,0,0
168,2020Q4,776,Male,1,0,0,0,1,0,0
169,2020Q4,1063,Male,1,0,0,0,0,1,0
170,2020Q4,89,Male,1,0,0,0,0,0,1
171,2020Q4,885,Female,0,1,0,0,1,0,0
172,2020Q4,945,Female,0,1,0,0,0,1,0
173,2020Q4,263,Female,0,1,0,0,0,0,1


In [79]:
# calculate the difference in Number_of_employees between the years 2011Q1 and 2020Q4
diff = df_subset.loc[df_subset["Year_Quarter"] == "2011Q1","Number_of_employees"].iloc[0] - \
       df_subset.loc[df_subset["Year_Quarter"] == "2020Q4","Number_of_employees"].iloc[0]
diff

126

In [80]:
# create a new column to indicate whether the difference is positive, negative, or zero
df_subset["diff_sign"] = df_subset["Number_of_employees"].apply(lambda x: "+" if x > diff else "-" if x < diff else "0")

In [81]:
df_subset["diff_sign"]

0      +
1      -
78     +
79     -
168    +
169    +
170    -
171    +
172    +
173    +
174    +
175    +
480    +
481    +
482    +
483    +
484    +
485    +
486    +
487    +
Name: diff_sign, dtype: object

In [82]:
# count the number of positive and negative differences
counts = df_subset["diff_sign"].value_counts()
counts

diff_sign
+    17
-     3
Name: count, dtype: int64

In [83]:
from scipy import stats

In [84]:
# calculate the p-value
p_value = stats.binom_test(counts, n=counts.sum(), p=0.5)

In [85]:
print(f"Test statistic: {counts}")
print(f"P-value: {p_value}")


Test statistic: diff_sign
+    17
-     3
Name: count, dtype: int64
P-value: 0.0025768280029296875


In [86]:
## Accroding to the pvalue 0.0025 < alpha(0.05) we reject the H0, 
## means there is no evidence to say that The median Number_of_employees per Year_Quarter in construction industry are the same for the years between 2011Q1 and 2020Q4.

## 2. Kruskal-Wallis H test

In [87]:
from scipy.stats import kruskal

In [88]:
## H0: there is no difference in weekly earnings in Quater1 to Q4
## H1: there is at least one of the Quarterly earnings has differences 

In [89]:
df.tail()

,Year_Quarter,Number_of_employees,gender,male,female,Construction,Clerical_support_workers,Skill_levels_high,Skill_level_medium,Skill_level_low
483,2011Q1,545,Female,0,1,0,0,1,0,0
484,2011Q1,918,Female,0,1,0,0,0,1,0
485,2011Q1,395,Female,0,1,0,0,0,0,1
486,2011Q1,145,Male,1,0,0,1,0,0,0
487,2011Q1,259,Female,0,1,0,1,0,0,0


In [90]:
df_subset1 = df[["Year_Quarter","Number_of_employees"]]
df_subset1.head()

,Year_Quarter,Number_of_employees
0,2020Q4,185
1,2020Q4,23
2,2020Q3,203
3,2020Q3,23
4,2020Q2,206


In [91]:
##df.to_csv("df_subset1", index=False)

In [92]:
# group the data by quarter
q1 = df.loc[df['Year_Quarter'] == '2015Q1']['Number_of_employees']
q2 = df.loc[df['Year_Quarter'] == '2015Q2']['Number_of_employees']
q3 = df.loc[df['Year_Quarter'] == '2015Q3']['Number_of_employees']
q4 = df.loc[df['Year_Quarter'] == '2015Q4']['Number_of_employees']


In [93]:
# perform the Kruskal-Wallis H test
stat, p_value = kruskal(q1, q2, q3, q4)

In [94]:
# display the results
print('Kruskal-Wallis H Test Results:')
print('Test Statistic: {:.4f}'.format(stat))
print('p-value: {:.4f}'.format(p_value))

Kruskal-Wallis H Test Results:
Test Statistic: 0.1504
p-value: 0.9852


In [95]:
if p_value < 0.05:
    print('There is a significant difference between the quarters.')
else:
    print('There is no significant difference between the quarters.')

There is no significant difference between the quarters.


In [96]:
## According to the results Pvalue 0.98 is > alpha(0.05) so we accept the H0,
## means There is no significant difference between the Skilled_operatives weely earnings in quarters 

## 3. U-Mann Whitman test

In [97]:
df2=df[['Skill_levels_high','Skill_level_low']]

In [98]:
df2.head()

,Skill_levels_high,Skill_level_low
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0


In [99]:
## H0: There is no difference Number of Skill_levels_high employees and Number of Skill_levels_low 
## H1: There is no difference Number of Skill_levels_high employees and Number of Skill_levels_low 

In [100]:
from scipy.stats import mannwhitneyu

In [101]:
mannwhitneyu(df2['Skill_levels_high'], df2['Skill_level_low'])

MannwhitneyuResult(statistic=119072.0, pvalue=1.0)

In [102]:
## according to the results pvalue 1.0 is > alpha(0.05) so we accept the H0,
## means there is no evidence to say that There is no difference Number of Skill_levels_high employees and Number of Skill_levels_low 

## Machine Learning Tasks